this script investigates the ratio of international colloboration funded exclusively by other country within each countries

In [1]:
import pandas as pd

In [2]:
pubs_path="../../data/nf_folder/Data/DerivedData/CleanedRawData/pub.pkl"
region_path="../../data/nf_folder/Data/AdditionalData/cntry_region.xlsx"

In [3]:
pubs_df=pd.read_pickle(pubs_path)
pubs_df.head()

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
0,60694041,2017,9,1,"[Canada, United States]","[United States, United States]","[United States, Canada]",[United States],1,1,0,0
1,48573156,2012,89,1,[France],Not-Funded,[France],Not-Funded,0,0,0,0
2,55323671,2015,89,1,"[France, Chile]","[Chile, Chile, Chile, Chile, Chile]","[Chile, France]",[Chile],1,1,0,0
3,57633115,2016,31,1,"[Switzerland, United States]",[Switzerland],"[Switzerland, United States]",[Switzerland],1,1,0,0
4,64831413,2018,99,1,[China],"[China, China, China, China, China, China]",[China],[China],1,0,0,0


In [4]:
intcoll_df = pubs_df[pubs_df.IntCol==1]
intcoll_df = intcoll_df[['id','year','author_distinct','funder_distinct']].explode('author_distinct')

In [5]:
def assign_type(author, funders):
    ftype = 'null'
    if funders=='Not-Funded':
        ftype = 'Not-Funded'
    elif author in funders:
        ftype = 'dome'
    else:
        ftype='fore'
    return ftype
intcoll_df['type']=intcoll_df.apply(lambda x:assign_type(x.author_distinct,x.funder_distinct), axis=1)

for each country, calculate the proportion of papers that are funded in each category

In [7]:
df = intcoll_df.groupby(by =['author_distinct','type']).size().reset_index(name='count')
df = df.pivot_table(index='author_distinct',columns='type',values='count',fill_value=0)
df = df.div(df.sum(axis=1), axis=0)

In [8]:
region=pd.read_excel(region_path)
region=region[['cntry','region2']]
region=region.rename(columns={'region2':'region'})
region=region.replace(to_replace={'region':{'EU':'Europe','Other Europe':'Europe'}})
region.head(2)

,cntry,region
0,Algeria,Africa
1,Angola,Africa


In [10]:
df = df.reset_index().merge(region, left_on='author_distinct',right_on='cntry')

In [12]:
df.groupby(['region'])['fore'].mean()

region
Africa           0.555880
Asia             0.433297
Europe           0.393249
North America    0.502058
Oceania          0.640800
South America    0.460163
Name: fore, dtype: float64